In [206]:
#libraries for classfication
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression #Regression
from sklearn.linear_model import LinearRegression #Regression
from sklearn.metrics import mean_squared_error, r2_score #Regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,PrecisionRecallDisplay
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,cross_val_score,StratifiedKFold, LeaveOneOut
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
#for precision recall curve
from sklearn.metrics import precision_recall_curve, roc_curve,matthews_corrcoef
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import label_binarize
from collections import Counter
from statsmodels.stats.contingency_tables import mcnemar
from scipy.stats import ttest_rel

PREVISUALIZACION DE LOS DATOS

In [207]:
def show_correlation_matrix(df):
    corr = df.corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, annot=True, fmt=".2f", square=True, cbar=True)
    plt.show()

In [208]:
#DATASET LOADING
#ESTE ES EL DATASET SIN MODIFICAR
df = pd.read_csv("B.HEALTH_classification.csv")

In [ ]:
#GRAFICA DEL DATASET
#GRAFICAMOS CANTIDAD DE DATOS POR CLASE
#0 es buena calidad 2 es mala calidad
sns.countplot(x='label', data=df)
plt.show()
print(df["label"].value_counts())
print(df.describe())

In [ ]:
#MOSTRAR MATRIZ DE CORRELACION
base_df = df.drop(columns=["subject","day"])
columns = [] #si queremos ver la matriz de correlacion sin alguna columna agregarlo aqui
show_correlation_matrix(base_df.drop(columns=columns))

 PREPARACION DE LOS DATOS PARA EL ENTRENAMIENTO Y VISUALIZACION DE CARACTERISTICAS

In [211]:
def normalize_data(X, how="minmax"):
    if how == "minmax":
        scaler = MinMaxScaler()
    elif how == "standard":
        scaler = StandardScaler()
    return scaler.fit_transform(X)

In [212]:
def apply_tsne(X, n_components=2, pca=False):
    if pca:
        reduccion = TSNE(n_components, init='pca')
    else:
        reduccion = TSNE(n_components)
    x_new = reduccion.fit_transform(X)
    return x_new

In [213]:
def plot_data_TSNE(x,y): #plot with TSNE and PCA to reduce dimensions
    fig, ax = plt.subplots(1, 1, figsize=(12, 5))
    
    x_new = apply_tsne(x, n_components=2, pca=True)
    tmp_df = pd.DataFrame(np.column_stack([x_new, y]))
    tmp_df.columns = ["x1", "x2", "Y"]
    
    sns.scatterplot(x="x1", y="x2", hue="Y", data=tmp_df, ax=ax)
    

In [214]:
#separate test and train data
def split_data(X, y, test_size=0.3):
    return train_test_split(X, y, test_size=test_size, random_state=1)

In [215]:
#REMOVEMOS COLUMNAS QUE NO QUEREMOS Y APLICAMOS SMOTE DE SER NECESARIO
remove_columns = ["subject","day", "label"] #remover columnas que no queremos para las variables
use_smote = True

X, Y = df.drop(columns=remove_columns), df["label"]

if use_smote:
    smote = SMOTE(sampling_strategy='minority')
    X, Y = smote.fit_resample(df.drop(columns=remove_columns), df["label"])


In [216]:
#NORMALIZAMOS LOS DATOS
X = normalize_data(X, how="minmax")

In [ ]:
#SEPARAMOS LOS DATOS DE ENTRENAMIENTO Y PRUEBA
#SI NO ESTA USANDO SMOTE PROCURAR 50% DE LA
X_train, X_test, y_train, y_test = split_data(X, Y, test_size=0.2)
print(y_train.value_counts())
sns.countplot(x=y_train)
plt.show()


In [ ]:
#PLOTTING DATA
plot_data_TSNE(X, Y)

ENTRENAMIENTO Y TUNNING DE MODELOS PARA CLASIFICACION

In [219]:
def select_model(model_name="DecisionTree"): 
    if model_name == "DecisionTree":
        return DecisionTreeClassifier()
    elif model_name == "SVM":
        return SVC()
    elif model_name == "RandomForest":
        return RandomForestClassifier()
    elif model_name == "XGBoost":
        return XGBClassifier()
    elif model_name == "MLP":
        return MLPClassifier()
    else:
        print("Invalid model name")
        return None

In [220]:
#comparing each model accuracy
def compare_models_accuracy(X_train, y_train, X_test, y_test):
    models = ["DecisionTree", "SVM", "RandomForest"]
    accuracies = []
    for model_name in models:
        model = select_model(model_name)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    return list(zip(models, accuracies))
        

In [221]:
def compare_models_accuracy(X_train, y_train, X_test, y_test):
    models = ["DecisionTree", "SVM", "RandomForest", "XGBoost"]
    accuracies = []
    for model_name in models:
        model = select_model(model_name)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    return list(zip(models, accuracies))

In [ ]:
#plotting comparison of models with no hyperparameter tuning
comparison_list = compare_models_accuracy(X_train, y_train, X_test, y_test)
print(comparison_list)


In [223]:
def parameter_tunning(model,parameters, method="grid", scoring_metric = "accuracy", cv = 5):
    if method == "grid":
        clf = GridSearchCV(model, param_grid=parameters, scoring=scoring_metric, cv=cv, n_jobs=-1)  
    # clf_svc = RandomizedSearchCV(clf_svc, param_distributions=parameters, scoring="accuracy", cv=5, n_jobs=-1, n_iter=15)
    else:
        clf = RandomizedSearchCV(model, param_distributions=parameters, scoring=scoring_metric, cv=cv, n_jobs=-1, n_iter=15)
    
    clf.fit(X_train, y_train)
    
    print("-----------------")
    print("Model", model.__class__.__name__)
    print("Best score : ", clf.best_score_)
    print("Best Parameters : ", clf.best_params_)
    print("-----------------")
    return clf

In [224]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    confusion = confusion_matrix(y_test, y_pred)
    print("Model", model.__class__.__name__)
    print("Accuracy: ", accuracy)
    print("F1 Score: ", f1)
    print("Confusion Matrix: ")
    print(confusion)
    return accuracy, f1, confusion

In [225]:
# parameters = {
#    "n_estimators":[i for i in range(1,200)],                   
#    "learning_rate":[0.1*i for i in range(1,10)],
#    "max_depth":[i for i in range(1,20)],                                            
# }
# useGridSearch = False
# parameter_tunning(select_model("XGBoost"), parameters, useGridSearch)

In [226]:
#DecisionTree parameters
# weights = np.linspace(0.0,0.99,20)
# print(weights)
# parameters = {
#     "criterion": ["gini", "entropy","log_loss"],
#     "max_depth": [i if i!=0 else None for i in range(1, 10)],
#     "max_features": ["auto", "sqrt", "log2"],
#     'class_weight': [{0:x, 1:1.0-x , 2:1.0 - x} for x in weights]
# }
# model = DecisionTreeClassifier()
# parameter_tunning(model, parameters, scoring_metric="f1_weighted")

In [227]:
# decision_tree = DecisionTreeClassifier(criterion="entropy", max_depth=9, max_features="sqrt")
# decision_tree.fit(X_train, y_train)
# y_pred = decision_tree.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy DecisionTree: ", accuracy)

In [228]:
# svm = SVC()
# #weights = np.linspace(0.0,0.99,200)
# parameters = {
#     "C": [0.1, 1, 10, 100],
#     "kernel": ["linear", "poly", "rbf", "sigmoid"], 
#     "gamma": [0.01*i for i in range(1, 100)],
#     #'class_weight': [{0:x, 1:1.0-x,2:1.0 - x} for x in weights]
# }
# parameter_tunning(svm, parameters,scoring_metric="f1_weighted")

In [229]:
# svm = SVC(C=10, degree=2, gamma=0.44, kernel="rbf")
# clf = svm.fit(X_train, y_train)
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy SVM: ", accuracy)

In [230]:

def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in"""
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

In [231]:
def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier."""
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [232]:
#APPLYTING TSNE TO VISUALIZE DATA WITH SVC
# plot_data_TSNE(X, Y)
# plt.show()
# X_tsne = apply_tsne(X, n_components=2, pca=True)
# X_train_tsne, X_test_tsne, y_train_tsne, y_test_tsne = split_data(X_tsne, Y)
# clf = svm.fit(X_train_tsne, y_train_tsne)

#  # Create a meshgrid for plotting
# X0, X1 = X_tsne[:, 0], X_tsne[:, 1]
# xx, yy = make_meshgrid(X0, X1)

# # Plot the decision boundaries
# fig, ax = plt.subplots()
# plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
# ax.scatter(X0, X1, c=Y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
# ax.set_xlabel('TSNE Component 1')
# ax.set_ylabel('TSNE Component 2')
# ax.set_title('SVM Decision Boundary')
# plt.show()



In [233]:
# random_f= RandomForestClassifier()

# parameters = {
#     "n_estimators": [50, 100, 200, 300],
#     "criterion": ["gini", "entropy","log_loss"],
#     "max_depth": [i if i!=0 else None for i in range(1, 10)],
#     "max_features": ["auto", "sqrt", "log2"],
#     'class_weight': [{0:x, 1:1.0-x , 2:1.0 - x} for x in weights]
# }
# parameter_tunning(random_f, parameters)

In [234]:
# random_f= RandomForestClassifier(criterion="entropy", max_depth=5, max_features="log2", n_estimators=50)
# random_f.fit(X_train, y_train)
# y_pred = random_f.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy RandomForest: ", accuracy)

In [235]:
# xgboost = XGBClassifier()
# parameters = {
#    "n_estimators":[i for i in range(1,200)],                   
#    "learning_rate":[0.1*i for i in range(1,10)],
#    "max_depth":[i for i in range(1,20)],
#    "max_features":[i for i in range(1,20)],
#    'class_weight': [{0:x, 1:1.0-x , 2:1.0 - x} for x in weights]
                                                  
# }
# parameter_tunning(xgboost, parameters, method="random", scoring_metric="f1_weighted")

In [236]:
# mlp_model = MLPClassifier()
# parameters = {
#     "hidden_layer_sizes": [(100,), (200,), (300,)],
#     "activation": ["identity", "logistic", "tanh", "relu"],
#     "solver": ["lbfgs", "sgd", "adam"],
#     "alpha": [0.0001, 0.001, 0.01, 0.1],
#     "learning_rate": ["constant", "invscaling", "adaptive"]
# }
# parameter_tunning(mlp_model, parameters)

In [237]:
# mlp_model = MLPClassifier(activation="relu", alpha=0.1, hidden_layer_sizes=(100,), learning_rate="invscaling", solver="adam")
# mlp_model.fit(X_train, y_train)
# y_pred = mlp_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy MLP: ", accuracy)

In [ ]:
#LISTA DE MODELOS Y PARAMETROS PARA TUNEAR
models = [DecisionTreeClassifier(),SVC(),RandomForestClassifier(),XGBClassifier()]
weights = np.linspace(0.0,0.9,10) #lista de pesos para el entrenamiento con datos desbalanceados
parameters = [
    {
        "criterion": ["gini", "entropy","log_loss"],
        "max_depth": [i if i!=0 else None for i in range(0, 10)],
        "max_features": ["auto", "sqrt", "log2"],
        'class_weight': [{2:x} for x in weights]
    },
    {
        "C": [0.1, 1, 10, 100],
        "kernel": ["linear", "poly", "rbf", "sigmoid"], 
        "gamma": [0.1*i for i in range(1, 100)],
        'class_weight': [{2:x} for x in weights]
    },
    {
        "n_estimators": [50, 100, 200, 300],
        "criterion": ["gini", "entropy","log_loss"],
        "max_depth": [i if i!=0 else None for i in range(0, 10)],
        "max_features": ["auto", "sqrt", "log2"],
        'class_weight': [{2:x} for x in weights]
    },
    {
        "n_estimators":[i for i in range(1,200)],                   
        "learning_rate":[0.1*i for i in range(1,10)],
        "max_depth":[i for i in range(1,20)],
        "max_features":[i for i in range(1,20)],
        'class_weight': [{2:x} for x in weights]
    },
]

#TUNEO DE HIPERPARAMETROS A CADA MODELO Y VALIDACION CRUZADA PARA CADA MODELO
#UTILIZAMOS TUNEO DE HIPERPARAMETROS CON VALIDACION CRUZADA PARA UN CRITERIO PRINCIPAL,
#DESPUES VERIFICAMOS EL MODELO CON CROSS VALI
best_models_with_scores = []
cv = StratifiedKFold(n_splits=5)
for i in range(len(models)):
    model = models[i]
    params = parameters[i]
    clf = parameter_tunning(model, params, method="random", scoring_metric="f1_weighted", cv=cv) #this shall return the best model with parameters

    best_models_with_scores.append((clf.best_estimator_, clf.best_score_, clf.best_params_))

best_models_with_scores    

ANALISIS Y COMPARACION DE MODELOS


In [ ]:
def decimal_to_time(decimal_time): ##calculo para pasar tiempo decimal a horas minutos y segundos
    """
    Convert a decimal time representation to HH:MM:SS format.
    
    Parameters:
    decimal_time (float): The decimal representation of time.
    
    Returns:
    str: The time in HH:MM:SS format.
    """
    total_seconds = int(decimal_time * 24 * 3600)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"


print(decimal_to_time(0.457638889)) 
print(decimal_to_time(0.707638889)) 

In [240]:
def graph_precision_recall_curve(y_test, y_score, n_classes=3):
    precision = dict()
    recall = dict()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                            y_score[:, i])
        plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))
        
    plt.xlabel("recall")
    plt.ylabel("precision")
    plt.legend(loc="best")
    plt.title("precision vs. recall curve")
    plt.show()

In [ ]:
# #classifiers with best hyperparameters
if use_smote:
# #SMOTE
    decision_tree = DecisionTreeClassifier(criterion="entropy", max_depth=5, max_features="log2")
    xgbModel = XGBClassifier(learning_rate=0.2, max_depth=19, n_estimators=154)
    svm = SVC(C=100, degree=2, gamma=0.09, kernel="rbf",probability=True)
    random_f= RandomForestClassifier(criterion="entropy", max_depth=6, max_features="sqrt", n_estimators=50)
    # mlp_model = MLPClassifier(activation="relu", alpha=0.01, hidden_layer_sizes=(200,), learning_rate="invscaling", solver="adam")
else:
# # #WEIGHTED
    decision_tree = DecisionTreeClassifier(criterion="entropy", max_depth=None, max_features="sqrt",class_weight={2: np.float64(0.7)})
    xgbModel = XGBClassifier(learning_rate=0.4, max_depth=2,n_estimators=38)
    #xgbModel = XGBClassifier(learning_rate=0.4, max_depth=2, max_features=11, n_estimators=38,class_weight={2: np.float64(0.7)})
    svm = SVC(C=0.1, gamma=1, kernel="poly",probability=True,class_weight={2:0.9}) 
    random_f= RandomForestClassifier(criterion="log_loss", max_depth=7, max_features="sqrt", n_estimators=50 , class_weight= {2: np.float64(0.9)} )
    # mlp_model = MLPClassifier(activation="relu", alpha=0.01, hidden_layer_sizes=(200,), learning_rate="invscaling", solver="adam")
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
random_f.fit(X_train, y_train)
xgbModel.fit(X_train, y_train)
# mlp_model.fit(X_train, y_train)
evaluate_model(decision_tree, X_test, y_test)
evaluate_model(svm, X_test, y_test)
evaluate_model(random_f, X_test, y_test)
evaluate_model(xgbModel, X_test, y_test)
# evaluate_model(mlp_model, X_test, y_test)

models = [decision_tree, svm, random_f, xgbModel]
#CROSS VALIDATION WITH TEST DATA
cross_val_scores = []
# cv = StratifiedKFold(n_splits=5)
cv = StratifiedKFold(n_splits=3)
for model in models:
    if use_smote:
        scores = cross_val_score(model, X_test, y_test, cv=cv, scoring="accuracy")
    else:
        scores = cross_val_score(model, X_test, y_test, cv=cv, scoring="f1_weighted")
    cross_val_scores.append((model.__class__.__name__,scores.mean(), scores.std()))

print("Cross Validation Scores")
for model, mean, std in cross_val_scores:
    print(f"Model: {model}, Mean: {mean}, Std: {std}") 

In [242]:
#WEIGHING CLASSES
# svm = SVC(probability=True)

# weights = np.linspace(0.0,0.99,200)
# param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}
# #Fitting grid search to the train data with 5 folds
# gridsearch_csv_results = parameter_tunning(svm, param_grid, method="grid")

# #Ploting the score for different values of weight
# sns.set_style('whitegrid')
# plt.figure(figsize=(12,8))
# weigh_data = pd.DataFrame({ 'score': gridsearch_csv_results['mean_test_score'], 'weight': (1- weights)})
# sns.lineplot(data=weigh_data, y='score', x='weight')
# plt.xlabel('Weight for class 1')
# plt.ylabel('F1 score')
# plt.xticks([round(i/10,1) for i in range(0,11,1)])
# plt.title('Scoring for different class weights', fontsize=24)

In [ ]:
#Testing model with best hyperparameters and weights or smote
# model = SVC(C=100, gamma=0.03, kernel="rbf",probability=True,class_weight={0: 0.5223618090452261, 1:0.4776381909547739, 2:0.4776381909547739}) #model without smote and applying weights
# #model = SVC(C=100, degree=2, gamma=0.09, kernel="rbf",probability=True) #model with smote
# evaluate_model(model,X_test, y_test)

# n_classes = 3
# Y_pr = label_binarize(Y, classes=[0, 1, 2])#esto es necesario para la curva de precision recall
# for model in best_models_with_scores:
#     y_score = model[0].predict_proba(X_test)
#     graph_precision_recall_curve(y_test, y_score, n_classes)

from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

def plot_roc_and_pr_curves(y_test, y_score, classes):
    n_classes = y_test.shape[1]

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Plot ROC curve
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], lw=2,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                       ''.format(classes[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

    # Compute Precision-Recall curve and average precision for each class
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_score[:, i])
        average_precision[i] = average_precision_score(y_test[:, i], y_score[:, i])

    # Compute micro-average Precision-Recall curve and average precision
    precision["micro"], recall["micro"], _ = precision_recall_curve(y_test.ravel(), y_score.ravel())
    average_precision["micro"] = average_precision_score(y_test, y_score, average="micro")

    # Plot Precision-Recall curve
    plt.figure()
    plt.plot(recall["micro"], precision["micro"],
             label='micro-average Precision-Recall curve (area = {0:0.2f})'
                   ''.format(average_precision["micro"]),
             color='gold', linestyle=':', linewidth=4)

    for i in range(n_classes):
        plt.plot(recall[i], precision[i], lw=2,
                 label='Precision-Recall curve of class {0} (area = {1:0.2f})'
                       ''.format(classes[i], average_precision[i]))

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.show()

# Example usage:
# Assuming y_test and y_score are your test labels and predicted scores respectively
# and classes is a list of class names or labels
y_score = svm.predict_proba(X_test)
Y_test_binarize = label_binarize(y_test, classes=[0, 1, 2])
classes = [0, 1, 2]
plot_roc_and_pr_curves(Y_test_binarize, y_score, classes)


In [ ]:
def mcnemar_test_between_models(models, X_test, y_test, alpha=0.05):
    n_models = len(models)
    wins = np.zeros(n_models)
    
    # Convert y_test to a numpy array
    y_test = np.array(y_test)
    
    # Get predictions for each model
    predictions = [model.predict(X_test) for model in models]
    
    # Perform pairwise McNemar tests
    for i in range(n_models):
        for j in range(i + 1, n_models):
            # Create contingency table
            contingency_table = np.zeros((2, 2))
            for k in range(len(y_test)):
                if predictions[i][k] == y_test[k] and predictions[j][k] == y_test[k]:
                    contingency_table[0, 0] += 1
                elif predictions[i][k] == y_test[k] and predictions[j][k] != y_test[k]:
                    contingency_table[0, 1] += 1
                elif predictions[i][k] != y_test[k] and predictions[j][k] == y_test[k]:
                    contingency_table[1, 0] += 1
                else:
                    contingency_table[1, 1] += 1
            
            # Perform McNemar test
            result = mcnemar(contingency_table, exact=True)
            p_value = result.pvalue
            
            # Print contingency table and p-value
            print(f"Comparison between model {i} and model {j}:")
            print("Contingency table:")
            print(contingency_table)
            print(f"P-value: {p_value}\n")
            
            # Update wins based on p-value
            if p_value < alpha:
                if contingency_table[0, 1] > contingency_table[1, 0]:
                    wins[i] += 1
                else:
                    wins[j] += 1
    
    # Determine the best model
    best_model_index = np.argmax(wins)
    best_model = models[best_model_index]
    
    return best_model, wins

# Example usage:
# Assuming you have trained models: model1, model2, model3
# and you have test data: X_test, y_test

models = [xgbModel, decision_tree, random_f, svm]
best_model, wins = mcnemar_test_between_models(models, X_test, y_test, alpha=0.05)

print("Best model:", best_model)
print("Wins:", wins)

In [245]:
def cross_val_5x2cv_paired_t_test(model1, model2, X, y):
    """
    Perform 5x2 cross-validated paired t-test to compare two models.
    
    Parameters:
    model1: First model to compare
    model2: Second model to compare
    X: Feature matrix
    y: Target vector
    
    Returns:
    t_stat: t-statistic
    p_value: p-value
    """
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    scores1 = []
    scores2 = []
    y = np.array(y)
    
    for i in range(5):
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            model1.fit(X_train, y_train)
            model2.fit(X_train, y_train)
            
            # evaluate_model(model1, X_test, y_test)
            # evaluate_model(model2, X_test, y_test)
            
            score1 = model1.score(X_test, y_test)
            score2 = model2.score(X_test, y_test)
            
            scores1.append(score1)
            scores2.append(score2)
    
    scores1 = np.array(scores1)
    scores2 = np.array(scores2)
    
    # Compute the differences in scores
    differences = scores1 - scores2
    
    # Compute the t-statistic and p-value
    t_stat, p_value = ttest_rel(scores1, scores2)
    
    return t_stat, p_value

In [ ]:
t_stat, p_value = cross_val_5x2cv_paired_t_test(xgbModel, random_f, X_test, y_test)
print("t-statistic:", t_stat)
print("p-value:", p_value)

In [ ]:
matthews_corrcoef(y_test, xgbModel.predict(X_test))


In [ ]:
matthews_corrcoef(y_test, svm.predict(X_test))

In [ ]:
matthews_corrcoef(y_test, random_f.predict(X_test))

In [ ]:
matthews_corrcoef(y_test, decision_tree.predict(X_test))